In [1]:
# create a csv file with as columns the database id and the sqlite file name, for the test databases
# the test databases are located in ./spider/test_database
# in this folder are located folders with the name of the database, and in each folder there is a sqlite file that can have any name

In [2]:
# extract the dataset into a csv file from the json file
import json
import csv

def extract_data(json_file, csv_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    with open(csv_file, 'w') as f:
        writer = csv.writer(f)
        writer.writerow(data[0].keys())
        for row in data:
            writer.writerow(row.values())

json_file = './spider/test_data/dev.json'
csv_file = './spider/test_spider.csv'
extract_data(json_file, csv_file)

In [3]:
def dbid_to_schema(db_id):
    with open('./spider/test_database/' + db_id + '/schema.sql') as f:
        lines = f.readlines()
    return lines

# get all db_ids in the ./spider/test_database folder
import os
db_ids_train = os.listdir('./spider/test_database')
db_ids_train = [db_id for db_id in db_ids_train if os.path.isdir('./spider/test_database/' + db_id)]

# remove from list all db_ids who do not have a schema.sql file, and put them in a new list
db_ids_no_schema = []
for db_id in db_ids_train:
    try:
        dbid_to_schema(db_id)
    except:
        db_ids_no_schema.append(db_id)
db_ids_train = [db_id for db_id in db_ids_train if db_id not in db_ids_no_schema]

# remove from the no_schema list all db_ids who do not have a .sql file, and put them in a new list
db_ids_no_sql = ['art_1', 'chinook_1', 'company_1', 'epinions_1', 'flight_4', 'icfp_1', 'small_bank_1', 'twitter_1', 'voter_1', 'world_1']
db_ids_no_schema = [db_id for db_id in db_ids_no_schema if db_id not in db_ids_no_sql]

# add outliers:
db_ids_outliers = ['college_1', 'college_2']
schema_outliers = ['TinyCollege.sql', 'TextBookExampleSchema.sql']

# remove all outliers from db_ids_no_schema
db_ids_no_schema = [db_id for db_id in db_ids_no_schema if db_id not in db_ids_outliers]

# remove the db_ids_no_sql from the test_spider.csv file
import pandas as pd
df = pd.read_csv('./spider/test_spider.csv')
df = df[~df['db_id'].isin(db_ids_no_sql)]
df = df[~df['db_id'].isin(db_ids_no_schema)]
df = df[~df['db_id'].isin(db_ids_outliers)]
df.to_csv('./spider/test_spider.csv', index=False)

In [4]:
# check if there are sqlite files in the database folders
db_ids_no_sqlite = []
for db_id in db_ids_train:
    try:
        db_id + '.sqlite' in os.listdir('./spider/test_database/' + db_id)
    except:
        db_ids_no_sqlite.append(db_id)
print(db_ids_no_sqlite)

[]


In [5]:
# create one csv file with two columns: db_id and schema
import csv
csv_file = './spider/test_database_schema.csv'

with open(csv_file, 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['db_id', 'schema'])
    for db_id in db_ids_no_schema:
        with open('./spider/test_database/' + db_id + '/' + db_id + '.sql') as f:
            schema = f.readlines()
            # remove insert statements
            # remove all INSERT lines from the schema except one per table
            kept_inserts = []
            used_tables = []
            for line in schema:
                if line.startswith('INSERT') or line.startswith('insert') or line.startswith(' insert') or line.startswith(' INSERT') or line.startswith('Insert') or line.startswith(' Insert') or line.startswith('\tINSERT') or line.startswith('\tinsert') or line.startswith('\tInsert'):
                    line_tokens = line.split()
                    table_name = line_tokens[2]
                    # remove all the line after the fourth occurence of ')'
                    if line.count(')') > 4:
                        first_find = line.find(')')
                        second_find = line.find(')', first_find + 1)
                        third_find = line.find(')', second_find + 1)
                        fourth_find = line.find(')', third_find + 1)
                        line = line[:line.find(')', fourth_find + 1)] + ')'              
                    if table_name in used_tables:
                        continue
                    else:
                        used_tables.append(table_name)
                        kept_inserts.append(line)
            schema = [line for line in schema if not line.startswith('INSERT') and not line.startswith('insert') 
                      and not line.startswith(' insert') and not line.startswith(' INSERT')
                      and not line.startswith('Insert') and not line.startswith(' Insert')
                      and not line.startswith('\tINSERT') and not line.startswith('\tinsert') and not line.startswith('\tInsert')]
            schema = schema + kept_inserts
            schema = ''.join(schema)
        writer.writerow([db_id, schema])
    for db_id in db_ids_train:
        schema = dbid_to_schema(db_id)
        # remove insert statements
        # remove all INSERT lines from the schema except one per table
        kept_inserts = []
        used_tables = []
        for line in schema:
            if line.startswith('INSERT') or line.startswith('insert') or line.startswith(' insert') or line.startswith(' INSERT') or line.startswith('Insert') or line.startswith(' Insert') or line.startswith('\tINSERT') or line.startswith('\tinsert') or line.startswith('\tInsert'):
                line_tokens = line.split()
                table_name = line_tokens[2]
                # remove all the line after the fourth occurence of ')'
                if line.count(')') > 4:
                    first_find = line.find(')')
                    second_find = line.find(')', first_find + 1)
                    third_find = line.find(')', second_find + 1)
                    fourth_find = line.find(')', third_find + 1)
                    line = line[:line.find(')', fourth_find + 1)] + ')'  
                if table_name in used_tables:
                    continue
                else:
                    used_tables.append(table_name)
                    kept_inserts.append(line)
        schema = [line for line in schema if not line.startswith('INSERT') and not line.startswith('insert') 
                    and not line.startswith(' insert') and not line.startswith(' INSERT')
                    and not line.startswith('Insert') and not line.startswith(' Insert')
                    and not line.startswith('\tINSERT') and not line.startswith('\tinsert') and not line.startswith('\tInsert')]
        schema = schema + kept_inserts
        schema = ''.join(schema)
        writer.writerow([db_id, schema])
    for db_id in db_ids_outliers:
        with open('./spider/test_database/' + db_id + '/' + schema_outliers[db_ids_outliers.index(db_id)]) as f:
            schema = f.readlines()
            # remove insert statements
            # remove all INSERT lines from the schema except one per table
            kept_inserts = []
            used_tables = []
            for line in schema:
                if line.startswith('INSERT') or line.startswith('insert') or line.startswith(' insert') or line.startswith(' INSERT') or line.startswith('Insert') or line.startswith(' Insert') or line.startswith('\tINSERT') or line.startswith('\tinsert') or line.startswith('\tInsert'):
                    line_tokens = line.split()
                    table_name = line_tokens[2]
                    # remove all the line after the fourth occurence of ')'
                    if line.count(')') > 4:
                        first_find = line.find(')')
                        second_find = line.find(')', first_find + 1)
                        third_find = line.find(')', second_find + 1)
                        fourth_find = line.find(')', third_find + 1)
                        line = line[:line.find(')', fourth_find + 1)] + ')'  
                    if table_name in used_tables:
                        continue
                    else:
                        used_tables.append(table_name)
                        kept_inserts.append(line)
            schema = [line for line in schema if not line.startswith('INSERT') and not line.startswith('insert') 
                    and not line.startswith(' insert') and not line.startswith(' INSERT')
                    and not line.startswith('Insert') and not line.startswith(' Insert')
                    and not line.startswith('\tINSERT') and not line.startswith('\tinsert') and not line.startswith('\tInsert')]
            schema = schema + kept_inserts
            schema = ''.join(schema)
        writer.writerow([db_id, schema])

In [6]:
# add insert statements to schema when it is missing

import pandas as pd
db_ids_not_insert = []
df = pd.read_csv('./spider/test_database_schema.csv')
db_ids = df['db_id'].tolist()
for db_id in db_ids:
    schema = df[df['db_id'] == db_id]['schema'].values[0]
    if ('INSERT' or 'insert' or 'Insert') not in schema:
        db_ids_not_insert.append(db_id)

import sqlite3

def get_values(db_id):
    conn = sqlite3.connect(f'./spider/test_database/{db_id}/{db_id}.sqlite')
    conn.text_factory = lambda b: b.decode(errors = 'ignore')
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
    values = []
    for table in tables:
        table_name = table[0]
        value = conn.execute(f"SELECT * FROM {table_name}").fetchall()[:2]
        values.append(value)
    conn.close()
    return tables, values

for db_id in db_ids_not_insert:
    tables, values = get_values(db_id)
    inserts = []
    for i, table in enumerate(tables):
        table_name = table[0]
        if len(values[i]) != 0:
            insert = f'INSERT INTO {table_name} VALUES {values[i]}\n'
            inserts.append(insert)
    inserts = '\n'.join(inserts)
    df.loc[df['db_id'] == db_id, 'schema'] = df.loc[df['db_id'] == db_id, 'schema'] + inserts
df.to_csv('./spider/test_database_schema.csv', index=False)

In [7]:
# create new csv files for the train and dev datasets with only db_id, query and question, but without the schema

def clean_csv(csv_file, new_csv_file):
    df = pd.read_csv(csv_file)
    new_df = pd.DataFrame(columns=['db_id', 'query', 'question'])
    for i in range(len(df)):
        sample = df.iloc[i].to_dict()
        sample = sample['db_id'], sample['query'], sample['question']
        new_df.loc[i] = sample
    new_df.to_csv(new_csv_file, index=False)

clean_csv('./spider/test_spider.csv', './spider/test_spider_clean.csv')

In [8]:
def sample_schema(sample):
    db_id = sample['db_id']
    with open('./spider/test_database/' + db_id + '/schema.sql') as f:
        lines = f.readlines()
    new_sample = {}
    # concatenate the schema to a single string
    lines = ' '.join(lines)
    new_sample['schema'] = lines
    new_sample['query'] = sample['query']
    new_sample['question'] = sample['question']
    new_sample['db_id'] = db_id
    return new_sample